In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#Definimos los filtros de fecha por los años y meses que se decida indicar
## Año inicial disponible de la estación
print("Digite el año inicial disponible de la estación")
inicial_year = input()
print(('El año inicial disponible de la estación: {}').format(inicial_year))

## Año final disponible de la estación
print("\nDigite el año final disponible de la estación")
final_year = input()
print(('El año final disponible de la estación: {}').format(final_year))

#Mes inicial disponible de la estación
print("\nDigite el mes inicial disponible de la estación, en números")
inicial_month =input()
print(('El mes inicial disponible de la estación: {}').format(inicial_month))

#Mes final disponible de la estación
print("\nDigite el mes final disponible de la estación, en números")
final_month =  input()
print(('El mes final disponible de la estación: {}').format(final_month))

filtro = []
for year in range(int(inicial_year), int(final_year) + 1):
    if year == int(inicial_year):
        for month in range(int(inicial_month), 13):
            if month < 10:
                filtro.append(str(inicial_year) + '0' + str(month)) 
            else:
                filtro.append(str(inicial_year) + str(month))
    elif year == int(final_year):
        for month in range(1, int(final_month) + 1):
            if month < 10:
                filtro.append(str(final_year) + '0' + str(month)) 
            else:
                filtro.append(str(final_year) + str(month))  
    else:               
        for month in range(1,13):
            if month < 10:
                filtro.append(str(year) + '0' + str(month))
            else:            
                filtro.append(str(year) + str(month))        

Digite el año inicial disponible de la estación
2015
El año inicial disponible de la estación: 2015

Digite el año final disponible de la estación
2020
El año final disponible de la estación: 2020

Digite el mes inicial disponible de la estación, en números
5
El mes inicial disponible de la estación: 5

Digite el mes final disponible de la estación, en números
4
El mes final disponible de la estación: 4


In [3]:
#Código de la estación
CODIGO = '112181'
#especificas los años disponibles en la estación
filtro
#definir el tipo de estación que es
estado = 'AUTOMATICA'
altura = '124'

In [4]:
table = []
dicc={'width':"100%", 'border':"1", 'class':"body01", 'bordercolor':"#999999", 
      'cellpadding':"0", 'cellspacing':"1", 'align':"center", 'id':"dataTable"}
for fecha in filtro:
    url = 'https://senamhi.gob.pe/mapas/mapa-estaciones-2/_dato_esta_tipo02.php?estaciones={}&CBOFiltro={}&t_e=M&estado={}&cod_old=&cate_esta=EMA&alt={}'.format(
          CODIGO,
          fecha,
          estado,
          altura)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table.append(soup.find('table',dicc))  

In [6]:
rows = []
for t in table:
    for i in t.find_all('tr',{'aling': "center"}):
        rows.append(i.text.replace(" ","").replace("\n", " ").strip().split()) 
## Creating a dataframe        
senhami = pd.DataFrame(rows[1:], columns = rows[0])
# Dropping duplicates
senhami = senhami.drop_duplicates(senhami.columns.to_list(),keep = 'first')
#Dropping headers
dup = []
for n in range(len(senhami)):
    if senhami.loc[n,].to_list() == senhami.columns.to_list():
        dup.append(n) 
senhami = senhami.drop(dup,0)
senhami.index = range(len(senhami.index))
senhami

,AÑO/MES/DÍA,HORA,TEMPERATURA(°C),PRECIPITACIÓN(mm/hora),HUMEDAD(%),DIRECCIONDELVIENTO(°),VELOCIDADDELVIENTO(m/s)
0,2015/05/01,00:00,22.3,0.0,75,80,0.0
1,2015/05/01,01:00,22.3,0.0,76,81,0.1
2,2015/05/01,02:00,23.0,0.0,75,214,1.3
3,2015/05/01,03:00,22.1,0.0,80,211,1.1
4,2015/05/01,04:00,21.6,0.0,81,212,2.2
...,...,...,...,...,...,...,...
42031,2020/04/25,08:00,20.6,0.0,82,256,1.3
42032,2020/04/25,09:00,22.0,0.0,75,194,2.0
42033,2020/04/25,10:00,25.4,0.0,61,200,1.1
42034,2020/04/25,11:00,23.7,0.0,68,199,4.5


In [8]:
senhami.to_csv('campo_de_marte.csv', index = False)